In [1]:
import sys
sys.path.append('../../30_data_tools/')

In [2]:
from pathlib import Path
from tqdm import tqdm
from PIL import Image
import numpy as np
import pickle
import random
import cv2

In [3]:
from helper import load_dotenv

In [4]:
dotenv = load_dotenv()

In [5]:
halftone_jpgs = list(dotenv['DATA_DIR'].glob('./*/halftone600dpi/*.4c.jpg'))
relevant_data = []

for jpg_path in tqdm(halftone_jpgs):
    mask_path = jpg_path.parent / jpg_path.name.replace('.4c.jpg','.masks.pkl')

    if mask_path.exists() and jpg_path.exists():
        relevant_data.append((jpg_path, mask_path))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 3912/3912 [00:00<00:00, 33408.23it/s]


In [7]:
test_images = random.choices(relevant_data, k=25)
test_data = []

for jpg_path, mask_path in test_images:
    with mask_path.open('rb') as pkl_file:
        masks = pickle.load(pkl_file)

    for m in random.choices(masks, k=min([len(masks),5])):
        test_data.append((jpg_path, m))

# Prüfung: Passen Bounding Box und Mask zusammen

In [9]:
errors = []

for _,m in test_data:
    diff_y = abs(m['mask'].shape[0] - m['bbox'][3])
    diff_x = abs(m['mask'].shape[1] - m['bbox'][2])

    if diff_x > 3 or diff_y > 3:
        errors.append(m)

In [10]:
len(errors)

0

# Prüfung: Passen Maske und Bild zusammen?

In [11]:
def merge_images( img1, img2 ):
    direction = 'column' if img1.size[0] > img1.size[1] else 'row'
    
    if direction == 'row':
        width = img1.size[0] + img2.size[0]
        height = max([img1.size[1], img2.size[1]])

        new_image = Image.new('RGB',(width, height), (250,250,250))
        new_image.paste(img1, (0,0))
        new_image.paste(img2, (img.size[0],0))
    else:
        width = max([img1.size[0], img2.size[0]])
        height = img1.size[1] + img2.size[1]

        new_image = Image.new('RGB',(width, height), (250,250,250))
        new_image.paste(img1, (0,0))
        new_image.paste(img2, (0,img.size[1]))

    if new_image.size[0] > 1000:
        new_image = new_image.resize((
            1000,
            int(new_image.size[1] * (1000 / new_image.size[0]))
        ))

    if new_image.size[1] > 1000:
        new_image = new_image.resize((
            int(new_image.size[0] * (1000 / new_image.size[1])),
            1000
        ))
    
    return new_image

In [12]:
i = 0

In [130]:
jpg_path, m = test_data[i]

img = Image.open(jpg_path).crop((
    m['bbox'][0],m['bbox'][1],
    m['bbox'][0]+m['bbox'][2],m['bbox'][1]+m['bbox'][3]
)).convert('RGB')
mask_img = Image.fromarray((m['mask'] * 255).astype('uint8')).convert('RGB')

print(f'{i+1}/{len(test_data)} - { jpg_path.parent.parent.name }/{ jpg_path.name }')

i = i+1

merge_images( img, mask_img )

IndexError: list index out of range